https://wikidocs.net/30708

In [27]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [28]:
#시간별로 vote 수
popular=pd.read_csv("/content/drive/MyDrive/Github/netflix_EDA/data/popular_des2.csv",index_col=0)
no_popular = pd.read_csv("/content/drive/MyDrive/Github/netflix_EDA/data/non_popular_des2.csv",index_col=0)

In [29]:
popular[:5]

,description_x
1,"On a photo shoot in Ghana, an American model s..."
7,A rookie cop with one day to prove himself to ...
8,When a powerful businesswoman’s political ambi...
11,"Bored with being the Lord of Hell, the devil r..."
19,After growing up amidst the gang wars of his h...


In [30]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [31]:
import numpy as np

popular_list = popular.values.tolist()
no_popular_list = no_popular.values.tolist()
popular_list[:5]

[['On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'],
 ["A rookie cop with one day to prove himself to a veteran LAPD narcotics officer receives a crash course in his mentor's questionable brand of justice."],
 ['When a powerful businesswoman’s political ambitions are threatened by her underworld connections, the ensuing power struggle could cost her everything.'],
 ['Bored with being the Lord of Hell, the devil relocates to Los Angeles, where he opens a nightclub and forms a connection with a homicide detective.'],
 ['After growing up amidst the gang wars of his hometown, Vincent forms an entrepreneurial squad of his own and ends up on the wrong side of the law.']]

In [32]:
import itertools
popular_list =list(itertools.chain.from_iterable(popular_list))
no_popular_list=list(itertools.chain.from_iterable(no_popular_list))

In [33]:
#텍스트 전처리
news_df = pd.DataFrame({'document':popular_list})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [34]:
news_df['clean_doc'][1]

'rookie with prove himself veteran lapd narcotics officer receives crash course mentor questionable brand justice'

In [35]:
import nltk
nltk.download('stopwords')
  
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
print(tokenized_doc[1])

['rookie', 'prove', 'veteran', 'lapd', 'narcotics', 'officer', 'receives', 'crash', 'course', 'mentor', 'questionable', 'brand', 'justice']


In [37]:
#3) TF-IDF 행렬 만들기
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [38]:
news_df['clean_doc'][1]

'rookie prove veteran lapd narcotics officer receives crash course mentor questionable brand justice'

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(802, 1000)

In [40]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

20

In [41]:
np.shape(svd_model.components_)


(20, 1000)

In [42]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('young', 0.3601), ('life', 0.2428), ('family', 0.22976), ('love', 0.19917), ('woman', 0.18984)]
Topic 2: [('father', 0.24235), ('murder', 0.17896), ('takes', 0.17027), ('family', 0.16988), ('crime', 0.16151)]
Topic 3: [('young', 0.40415), ('woman', 0.32557), ('father', 0.20479), ('future', 0.11205), ('marriage', 0.10793)]
Topic 4: [('family', 0.52176), ('returns', 0.19215), ('home', 0.18371), ('years', 0.09788), ('long', 0.09483)]
Topic 5: [('murder', 0.26771), ('daughter', 0.22277), ('mother', 0.1978), ('woman', 0.14768), ('missing', 0.14764)]
Topic 6: [('father', 0.29014), ('family', 0.2596), ('village', 0.22857), ('parents', 0.16855), ('daughter', 0.12873)]
Topic 7: [('death', 0.27617), ('finds', 0.26599), ('home', 0.16821), ('friends', 0.13836), ('father', 0.13564)]
Topic 8: [('true', 0.31181), ('based', 0.20916), ('story', 0.19095), ('india', 0.18615), ('friends', 0.1577)]
Topic 9: [('true', 0.26103), ('story', 0.16731), ('home', 0.16693), ('based', 0.16633), ('falls', 

In [43]:
tokenized_doc[:5]

0    [photo, shoot, ghana, american, model, slips, ...
1    [rookie, prove, veteran, lapd, narcotics, offi...
2    [powerful, businesswoman, political, ambitions...
3    [bored, lord, hell, devil, relocates, angeles,...
4    [growing, amidst, gang, wars, hometown, vincen...
Name: clean_doc, dtype: object

In [44]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0


[(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [45]:
print(dictionary[66])

attempts


In [46]:
len(dictionary)

4238

In [47]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

(0, '0.009*"young" + 0.007*"family" + 0.006*"revenge" + 0.006*"gangster"')
(1, '0.009*"girl" + 0.006*"lives" + 0.006*"city" + 0.006*"veera"')
(2, '0.011*"life" + 0.006*"home" + 0.006*"returns" + 0.005*"high"')
(3, '0.014*"father" + 0.009*"young" + 0.008*"life" + 0.006*"daughter"')
(4, '0.008*"family" + 0.006*"mission" + 0.005*"four" + 0.005*"woman"')
(5, '0.012*"three" + 0.008*"college" + 0.006*"lives" + 0.006*"friends"')
(6, '0.009*"young" + 0.007*"life" + 0.007*"daughter" + 0.005*"grandfather"')
(7, '0.008*"love" + 0.008*"parents" + 0.007*"young" + 0.006*"daughter"')
(8, '0.008*"family" + 0.008*"life" + 0.006*"drug" + 0.005*"crime"')
(9, '0.009*"crime" + 0.008*"father" + 0.008*"framed" + 0.008*"town"')
(10, '0.011*"young" + 0.011*"find" + 0.010*"father" + 0.008*"village"')
(11, '0.017*"young" + 0.010*"girl" + 0.008*"world" + 0.007*"choice"')
(12, '0.011*"young" + 0.010*"love" + 0.010*"years" + 0.007*"takes"')
(13, '0.011*"life" + 0.008*"young" + 0.006*"family" + 0.006*"detective"')
(

In [48]:
print(ldamodel.print_topics())

[(0, '0.009*"young" + 0.007*"family" + 0.006*"revenge" + 0.006*"gangster" + 0.006*"become" + 0.006*"child" + 0.005*"girl" + 0.005*"past" + 0.005*"begins" + 0.005*"dark"'), (1, '0.009*"girl" + 0.006*"lives" + 0.006*"city" + 0.006*"veera" + 0.004*"save" + 0.004*"home" + 0.004*"best" + 0.004*"finds" + 0.004*"wedding" + 0.004*"teenage"'), (2, '0.011*"life" + 0.006*"home" + 0.006*"returns" + 0.005*"high" + 0.005*"find" + 0.005*"strange" + 0.004*"work" + 0.003*"city" + 0.003*"finds" + 0.003*"wife"'), (3, '0.014*"father" + 0.009*"young" + 0.008*"life" + 0.006*"daughter" + 0.006*"love" + 0.005*"fall" + 0.005*"years" + 0.005*"stop" + 0.005*"asks" + 0.005*"troubled"'), (4, '0.008*"family" + 0.006*"mission" + 0.005*"four" + 0.005*"woman" + 0.005*"life" + 0.005*"love" + 0.005*"make" + 0.005*"home" + 0.004*"dreams" + 0.004*"first"'), (5, '0.012*"three" + 0.008*"college" + 0.006*"lives" + 0.006*"friends" + 0.006*"death" + 0.006*"truth" + 0.005*"young" + 0.005*"struggling" + 0.005*"different" + 0.005

In [49]:
!pip install pyLDAvis

# popular movie topic classification

In [50]:
# !pip install pyLDAvis
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7     -0.100647  0.106735       1        1  6.666371
15     0.053979 -0.055732       2        1  6.503887
0     -0.038762 -0.051427       3        1  5.730927
8     -0.025409 -0.085547       4        1  5.584131
4     -0.087773 -0.093755       5        1  5.477767
14    -0.106927  0.054045       6        1  5.376971
9     -0.048644 -0.071803       7        1  5.336465
6      0.083590  0.003908       8        1  5.280457
12     0.115739  0.013980       9        1  5.261281
10     0.094481  0.037970      10        1  5.219238
16     0.011204 -0.006282      11        1  5.212782
18     0.059438 -0.059850      12        1  4.933764
1     -0.011670  0.141676      13        1  4.699044
5     -0.033965 -0.034329      14        1  4.381115
2      0.005230  0.011030      15        1  4.332873
19     0.025858  0.025630      16        1  4.302093
3      0.002357  0.027992      17        1  4.231602
13    -0.036388 -0.004111      18        1  4.045631
11     0.006117 -0.000162      19        1  3.765937
17     0.032192  0.040032      20        1  3.657662, topic_info=          Term       Freq      Total Category  logprob  loglift
716     father  36.000000  36.000000  Default  30.0000  30.0000
593      woman  37.000000  37.000000  Default  29.0000  29.0000
609       girl  27.000000  27.000000  Default  28.0000  28.0000
212     family  52.000000  52.000000  Default  27.0000  27.0000
238     murder  27.000000  27.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
129    defends   1.346673   3.505396  Topic20  -5.6582   2.3517
177      india   1.346673  19.410796  Topic20  -5.6582   0.6402
515  befriends   1.346673   5.595843  Topic20  -5.6582   1.8840
574       deep   1.346673   4.144717  Topic20  -5.6582   2.1841
685  childhood   1.346673  10.686011  Topic20  -5.6582   1.2370

[1393 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2020      1  0.899561  abandoned
3400     18  0.484928       able
932       3  0.691794     abrupt
2057     20  0.731499   abstract
3423     20  0.731499     accept
...     ...       ...        ...
594      12  0.138405    younger
594      18  0.138405    younger
2436      3  0.689136      youth
2436      9  0.344568      youth
1171      4  0.693351    zombies

[3066 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 16, 1, 9, 5, 15, 10, 7, 13, 11, 17, 19, 2, 6, 3, 20, 4, 14, 12, 18])

In [51]:
!pip install pandas --upgrade

In [52]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7     -0.100647  0.106735       1        1  6.666371
15     0.053979 -0.055732       2        1  6.503887
0     -0.038762 -0.051427       3        1  5.730927
8     -0.025409 -0.085547       4        1  5.584131
4     -0.087773 -0.093755       5        1  5.477767
14    -0.106927  0.054045       6        1  5.376971
9     -0.048644 -0.071803       7        1  5.336465
6      0.083590  0.003908       8        1  5.280457
12     0.115739  0.013980       9        1  5.261281
10     0.094481  0.037970      10        1  5.219238
16     0.011204 -0.006282      11        1  5.212782
18     0.059438 -0.059850      12        1  4.933764
1     -0.011670  0.141676      13        1  4.699044
5     -0.033965 -0.034329      14        1  4.381115
2      0.005230  0.011030      15        1  4.332873
19     0.025858  0.025630      16        1  4.302093
3      0.002357  0.027992      17        1  4.231602
13    -0.036388 -0.004111      18        1  4.045631
11     0.006117 -0.000162      19        1  3.765937
17     0.032192  0.040032      20        1  3.657662, topic_info=          Term       Freq      Total Category  logprob  loglift
716     father  36.000000  36.000000  Default  30.0000  30.0000
593      woman  37.000000  37.000000  Default  29.0000  29.0000
609       girl  27.000000  27.000000  Default  28.0000  28.0000
212     family  52.000000  52.000000  Default  27.0000  27.0000
238     murder  27.000000  27.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
129    defends   1.346673   3.505396  Topic20  -5.6582   2.3517
177      india   1.346673  19.410796  Topic20  -5.6582   0.6402
515  befriends   1.346673   5.595843  Topic20  -5.6582   1.8840
574       deep   1.346673   4.144717  Topic20  -5.6582   2.1841
685  childhood   1.346673  10.686011  Topic20  -5.6582   1.2370

[1393 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2020      1  0.899561  abandoned
3400     18  0.484928       able
932       3  0.691794     abrupt
2057     20  0.731499   abstract
3423     20  0.731499     accept
...     ...       ...        ...
594      12  0.138405    younger
594      18  0.138405    younger
2436      3  0.689136      youth
2436      9  0.344568      youth
1171      4  0.693351    zombies

[3066 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 16, 1, 9, 5, 15, 10, 7, 13, 11, 17, 19, 2, 6, 3, 20, 4, 14, 12, 18])